# Notebook 5 : Web scraping

In [1]:
# Décommenter la ligne suivante pour installer lxml (nécessaire pour read_html)
# %pip install lxml

In [2]:
import pandas as pd
import requests

from bs4 import BeautifulSoup

## Bonsaïs

Le site [Umi Zen Bonsai](https://umizenbonsai.com/) est une boutique de vente en ligne dédiée aux bonsaïs. Les conifères sont disponible sur la page web [https://umizenbonsai.com/shop/bonsai/coniferes/](https://umizenbonsai.com/shop/bonsai/coniferes/). Comme beaucoup d'autres sites, l'information est organisée en blocs dans lesquels il est possible de récupérer des données.

Pour scraper ce type de site, le processus consiste à capturer les blocs dans un premier temps, puis à en extraire les données.

1. Récupérer le contenu de la page avec `requests` et passer le résultat au parser de `BeautifulSoup`.

In [3]:
url_bonsai = "https://umizenbonsai.com/shop/bonsai/coniferes/"

r_bonsai = requests.get(url_bonsai)

if r_bonsai.status_code != 200:
    print(f"Erreur {r_bonsai.status_code}")

soup_bonsai = BeautifulSoup(r_bonsai.text, "html.parser")

2. Écrire un sélecteur CSS pour capturer les éléments `li` qui contiennent les blocs correspondants aux bonsaïs. Vérifier sur le site que le nombre de bonsaïs affichés correspond.

In [4]:
# Les éléments li à récupérer ont tous une classe "entry".
selector_bonsai = "li.entry"
bonsai_list = soup_bonsai.select(selector_bonsai)

print(f"{len(bonsai_list)} bonsaïs")

9 bonsaïs


3. Écrire une fonction qui prend un bloc de la liste précédente et retourne un tuple contenant le nom, le prix et le lien de description du bonsaï.

In [5]:
def bonsai_info(bonsai):
    # Toutes les données sont dans une sous-liste à puce ul
    # Le sélecteur CSS est relatif à l'élément li transmis
    data = bonsai.select_one("div ul")

    # Nom et lien du bonsaï
    titre = data.select_one("li.title h2 a")
    nom = titre.text
    url = titre.attrs["href"]

    # Prix du bonsaï
    prix = data.select_one("li.price-wrap span span bdi").text

    return (nom, prix, url)

bonsai_info(bonsai_list[0])

('Bonsai Cyprès du Japon – 33cm',
 '140.00€',
 'https://umizenbonsai.com/vente/bonsai-cypres-du-japon/')

4. Utiliser les deux questions précédentes pour construire un dataframe contenant les données des bonsaïs.

In [6]:
bonsais_data = [bonsai_info(bonsai) for bonsai in bonsai_list]
bonsais = pd.DataFrame(
    bonsais_data,
    columns=["Nom", "Prix", "URL"]
)

bonsais

,Nom,Prix,URL
0,Bonsai Cyprès du Japon – 33cm,140.00€,https://umizenbonsai.com/vente/bonsai-cypres-d...
1,Bonsai Genévrier de Phénicie – 59cm,250.00€,https://umizenbonsai.com/vente/bonsai-genevrie...
2,Bonsai Genévrier de Phénicie – 67cm,370.00€,https://umizenbonsai.com/vente/bonsai-genevrie...
3,Bonsai If Commun – 58cm,"1,190.00€",https://umizenbonsai.com/vente/bonsai-if-commu...
4,Bonsai If Commun – 85cm,650.00€,https://umizenbonsai.com/vente/bonsai-if-commun/
5,Bonsai If du Japon – 50cm,750.00€,https://umizenbonsai.com/vente/bonsai-if-du-ja...
6,Bonsai If du Japon – 63cm,"1,290.00€",https://umizenbonsai.com/vente/bonsai-if-japon/
7,Bonsai Pin Sylvestre – 38cm,440.00€,https://umizenbonsai.com/vente/bonsai-pin-sylv...
8,Bonsai Pin Sylvestre – 65cm,370.00€,https://umizenbonsai.com/vente/bonsai-pin-sylv...


5. (*Bonus*) Écrire une fonction pour récupérer la provenance, le feuilage et les dimension du bonsaï à partir du lien de description. Utiliser cette fonction pour ajouter des colonnes au dataframe précédent

In [7]:
def bonsai_details(url):
    r = requests.get(url)
    if r.status_code != 200:
        print(f"Erreur {r.status_code}")

    soup = BeautifulSoup(r.text, "html.parser")
    data = soup.select("div.elementor-widget-container p")

    # L'attribut stripped_strings permet de découper les retours à la ligne <br/>
    dim = list(data[5].stripped_strings)
    
    return pd.Series({
        "Provenance": data[0].text,
        "Feuillage": data[1].text,
        "Nebari": dim[0],
        "Hauteur": dim[1],
    })

# Avec axis=1, la fonction concat accumule les colonnes de deux dataframes
pd.concat(
    [
        bonsais,
        bonsais.apply(lambda row: bonsai_details(row.URL), axis=1)
    ],
    axis=1,
)

,Nom,Prix,URL,Provenance,Feuillage,Nebari,Hauteur
0,Bonsai Cyprès du Japon – 33cm,140.00€,https://umizenbonsai.com/vente/bonsai-cypres-d...,Japon,Persistant,Mi-ombre.,A cultiver en extérieur toute l’année
1,Bonsai Genévrier de Phénicie – 59cm,250.00€,https://umizenbonsai.com/vente/bonsai-genevrie...,Yamadori France,Persistant,Soleil.,A cultiver en extérieur toute l’année
2,Bonsai Genévrier de Phénicie – 67cm,370.00€,https://umizenbonsai.com/vente/bonsai-genevrie...,Yamadori France,Persistant,Soleil.,A cultiver en extérieur toute l’année
3,Bonsai If Commun – 58cm,"1,190.00€",https://umizenbonsai.com/vente/bonsai-if-commu...,Yamadori France,"Persistant. Très toxique, ne pas ingérer",Mi-ombre.,A cultiver en extérieur
4,Bonsai If Commun – 85cm,650.00€,https://umizenbonsai.com/vente/bonsai-if-commun/,Yamadori France,"Persistant. Très toxique, ne pas ingérer",Mi-ombre.,A cultiver en extérieur toute l’année
5,Bonsai If du Japon – 50cm,750.00€,https://umizenbonsai.com/vente/bonsai-if-du-ja...,Japon,"Persistant. Très toxique, ne pas ingérer",Mi-ombre.,A cultiver en extérieur toute l’année
6,Bonsai If du Japon – 63cm,"1,290.00€",https://umizenbonsai.com/vente/bonsai-if-japon/,Japon,"Persistant. Très toxique, ne pas ingérer",Mi-ombre.,A cultiver en extérieur toute l’année
7,Bonsai Pin Sylvestre – 38cm,440.00€,https://umizenbonsai.com/vente/bonsai-pin-sylv...,Yamadori France,Persistant,Soleil.,A cultiver en extérieur toute l’année
8,Bonsai Pin Sylvestre – 65cm,370.00€,https://umizenbonsai.com/vente/bonsai-pin-sylv...,Yamadori France,Persistant,Soleil.,A cultiver en extérieur toute l’année


## Trampoline

Le trampoline est un sport olympique depuis les jeux de Sydney en 2000. La page suivante contient les listes des hommes et des femmes ayant obtenu une médaille olympique dans cette discipline :
[https://fr.wikipedia.org/wiki/Liste_des_m%C3%A9daill%C3%A9s_olympiques_au_trampoline](https://fr.wikipedia.org/wiki/Liste_des_m%C3%A9daill%C3%A9s_olympiques_au_trampoline)

Un tableau est contenu dans un élément `table` avec des balises pour les lignes `tr`, pour les colonnes `th`, pour les cellules `td`, ... Cela peut être fastidieux à scraper et très répétitif. Heureusement, Pandas propose la fonction `read_html` pour récupérer des tableaux sous forme de dataframes à partir d'une page web.

1. Utiliser la fonction `read_html` de Pandas sur la page des médaillés olympiques au trampoline. Combien de dataframes sont récupérés ?

In [8]:
trampoline_url = "https://fr.wikipedia.org/wiki/Liste_des_m%C3%A9daill%C3%A9s_olympiques_au_trampoline"
trampoline_dfs = pd.read_html(trampoline_url)

print(f"{len(trampoline_dfs)} dataframes")

6 dataframes


2. Extraire de la liste précédente les dataframes des médailles masculines et féminines.

In [9]:
trampoline_homme = trampoline_dfs[0]
trampoline_femme = trampoline_dfs[1]

3. À partir de ces dataframes, compter combien chaque pays a reçu de médailles d'or, d'argent et de bronze.

In [10]:
# L'expression régulière "\((.*)\)" correspond au texte entre parenthèses.
# La méthode value_count compte les occurences des valeurs distinctes.
# La méthode add permet de remplacer les données manquantes par des zéros.

# Médailles Or
or_homme = trampoline_homme.Or.str.extract("\((.*)\)").value_counts()
or_femme = trampoline_femme.Or.str.extract("\((.*)\)").value_counts()
or_medailles = or_homme.add(or_femme, fill_value=0)

or_medailles

AIN    1.0
BLR    2.0
CAN    2.0
CHN    4.0
GBR    1.0
GER    1.0
RUS    2.0
UKR    1.0
dtype: float64

In [11]:
# Médailles Argent
argent_homme = trampoline_homme.Argent.str.extract("\((.*)\)").value_counts()
argent_femme = trampoline_femme.Argent.str.extract("\((.*)\)").value_counts()
argent_medailles = argent_homme.add(argent_femme, fill_value=0)

argent_medailles

AIN    1.0
AUS    1.0
CAN    3.0
CHN    5.0
GBR    1.0
RUS    2.0
UKR    1.0
dtype: float64

In [12]:
# Médailles Bronze
bronze_homme = trampoline_homme.Bronze.str.extract("\((.*)\)").value_counts()
bronze_femme = trampoline_femme.Bronze.str.extract("\((.*)\)").value_counts()
bronze_medailles = bronze_homme.add(bronze_femme, fill_value=0)

bronze_medailles

CAN    3.0
CHN    7.0
GBR    1.0
GER    1.0
NZL    1.0
UZB    1.0
dtype: float64

4. (*Bonus*) Construire un dataframe contenant, pour chaque pays, le nombre de médailles d'or, d'argent et de bronze ainsi que le nombre total de médailles. Classer ce dataframe dans l'ordre usuel en fonction d'abord du nombre de médailles d'or, puis du nombre de médailles d'argent et enfin du nombre de médailles de bronze. Comparer le résultat avec le tableau des médailles sur la page [https://fr.wikipedia.org/wiki/Trampoline_aux_Jeux_olympiques](https://fr.wikipedia.org/wiki/Trampoline_aux_Jeux_olympiques).

In [13]:
# Concaténation des colonnes
medailles = (
    pd.DataFrame(
        {
            "Or": or_medailles,
            "Argent": argent_medailles,
            "Bronze": bronze_medailles,
        }
    )
    .rename_axis("Pays")
    .fillna(0)
    .assign(Total=lambda row: row.Or + row.Argent + row.Bronze)
    .sort_values(by=["Or", "Argent", "Bronze"], ascending=False)
    .astype(int)
)

medailles

,Or,Argent,Bronze,Total
Pays,,,,
CHN,4,5,7,16
CAN,2,3,3,8
RUS,2,2,0,4
BLR,2,0,0,2
GBR,1,1,1,3
AIN,1,1,0,2
UKR,1,1,0,2
GER,1,0,1,2
AUS,0,1,0,1


## Cate Blanchett

Dans le cours, nous avons essayé de trouver avec quels acteurs Cate Blanchett a joué le plus au cours des années 2000. Pour cela, nous avons récupéré la liste des pages Wikipedia des films où elle tient un rôle avec le code suivant :

In [14]:
url_wikipedia = "https://fr.wikipedia.org"
url_blanchett = url_wikipedia + "/wiki/Cate_Blanchett"

r_blanchett = requests.get(url_blanchett)
assert r_blanchett.status_code == 200, f"Erreur {r_blanchett.status_code}"

soup_blanchett = BeautifulSoup(r_blanchett.text, "html.parser")

selector_films = "#mw-content-text div ul:nth-of-type(3) li i a"
films_blanchett = soup_blanchett.select(selector_films)

films_data = [
    {
        "titre": film.attrs["title"],
        "url_wikipedia": url_wikipedia + film.attrs["href"]
    }
    for film in films_blanchett
    if not (
        film.attrs.get("class") == ["new"] # Film sans page
        or film.attrs["title"] == "Galadriel" # Mauvais lien
    )
]

films = pd.DataFrame(films_data)

films

,titre,url_wikipedia
0,Les Larmes d'un homme,https://fr.wikipedia.org/wiki/Les_Larmes_d%27u...
1,Intuitions,https://fr.wikipedia.org/wiki/Intuitions
2,"Bandits (film, 2001)","https://fr.wikipedia.org/wiki/Bandits_(film,_2..."
3,Le Seigneur des anneaux : La Communauté de l'a...,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...
4,Charlotte Gray,https://fr.wikipedia.org/wiki/Charlotte_Gray
5,Terre Neuve (film),https://fr.wikipedia.org/wiki/Terre_Neuve_(film)
6,"Heaven (film, 2002)","https://fr.wikipedia.org/wiki/Heaven_(film,_2002)"
7,Le Seigneur des anneaux : Les Deux Tours,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...
8,Veronica Guerin (film),https://fr.wikipedia.org/wiki/Veronica_Guerin_...
9,Coffee and Cigarettes,https://fr.wikipedia.org/wiki/Coffee_and_Cigar...


Le sélecteur CSS que nous avons utilisé ne permettait pas d'obtenir la réponse à notre question car il ne capturait pas toutes les listes d'acteurs (organisation différente pour *Coffee and Cigarettes*, double colonne pour *Aviator*, ...). En effet, les pages Wikipedia des films ne sont pas uniformes et il n'est pas possible d'extraire la distribution de tous les films avec le même sélecteur.

Pour remédier à cela, nous proposons ici d'aller scraper la liste des acteurs sur le site [TMDB](https://www.themoviedb.org/) (*The Movie Database*) dont les pages obéissent toutes à la même organisation. Les pages Wikipedia relatives à des films contiennent toutes un lien externe vers ce site.

1. Pour chaque film, scraper la page Wikipedia pour récupérer le lien vers la page TMDB associée et déduire le lien du casting complet qui ser ajouté le dans une nouvelle colonne du dataframe `films`.

In [15]:
def url_tmdb_casting(url_wikipedia):
    # Les liens externes ont tous la classe "external text"
    selector_lien_externe = "a[class='external text']"

    r_film = requests.get(url_wikipedia)
    assert r_film.status_code == 200, f"Erreur {r_film.status_code}"
    soup_film = BeautifulSoup(r_film.text, "html.parser")
    liens = soup_film.select(selector_lien_externe)
    liens_tmdb = [
        lien.attrs["href"]
        for lien in liens
        if lien.text == "The Movie Database"
    ]
    assert len(liens_tmdb) == 1, "Erreur de lien TMDB"
    # Il faut ajouter /cast au lien TMDB pour le casting
    return liens_tmdb[0] + "/cast"

films["url_tmdb_casting"] = films.url_wikipedia.apply(url_tmdb_casting)

films

,titre,url_wikipedia,url_tmdb_casting
0,Les Larmes d'un homme,https://fr.wikipedia.org/wiki/Les_Larmes_d%27u...,https://www.themoviedb.org/movie/29572/cast
1,Intuitions,https://fr.wikipedia.org/wiki/Intuitions,https://www.themoviedb.org/movie/2046/cast
2,"Bandits (film, 2001)","https://fr.wikipedia.org/wiki/Bandits_(film,_2...",https://www.themoviedb.org/movie/3172/cast
3,Le Seigneur des anneaux : La Communauté de l'a...,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...,https://www.themoviedb.org/movie/120/cast
4,Charlotte Gray,https://fr.wikipedia.org/wiki/Charlotte_Gray,https://www.themoviedb.org/movie/12660/cast
5,Terre Neuve (film),https://fr.wikipedia.org/wiki/Terre_Neuve_(film),https://www.themoviedb.org/movie/6440/cast
6,"Heaven (film, 2002)","https://fr.wikipedia.org/wiki/Heaven_(film,_2002)",https://www.themoviedb.org/movie/10575/cast
7,Le Seigneur des anneaux : Les Deux Tours,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...,https://www.themoviedb.org/movie/121/cast
8,Veronica Guerin (film),https://fr.wikipedia.org/wiki/Veronica_Guerin_...,https://www.themoviedb.org/movie/10629/cast
9,Coffee and Cigarettes,https://fr.wikipedia.org/wiki/Coffee_and_Cigar...,https://www.themoviedb.org/movie/883/cast


2. La liste des acteurs d'un film se présente comme une liste ordonnée `ol` dans les pages TMDB. Scraper les pages de casting pour ajouter la liste des acteurs de chaque film dans une nouvelle colonne du dataframe `films`.

In [18]:
def list_acteurs(url):
    r_casting = requests.get(url)
    assert r_casting.status_code == 200, f"Erreur {r_casting.status_code}"
    soup_casting = BeautifulSoup(r_casting.text, "html.parser")
    # Le combinateur d'enfant direct '>' permet de limiter le sélecteur au casting
    selector_casting = "section.panel:nth-of-type(1) ol li div div p a"
    return [acteur.text for acteur in soup_casting.select(selector_casting)]

films["acteurs"] = films.url_tmdb_casting.apply(list_acteurs)

films

,titre,url_wikipedia,url_tmdb_casting,acteurs
0,Les Larmes d'un homme,https://fr.wikipedia.org/wiki/Les_Larmes_d%27u...,https://www.themoviedb.org/movie/29572/cast,"[Christina Ricci, Johnny Depp, Cate Blanchett,..."
1,Intuitions,https://fr.wikipedia.org/wiki/Intuitions,https://www.themoviedb.org/movie/2046/cast,"[Cate Blanchett, Giovanni Ribisi, Keanu Reeves..."
2,"Bandits (film, 2001)","https://fr.wikipedia.org/wiki/Bandits_(film,_2...",https://www.themoviedb.org/movie/3172/cast,"[Bruce Willis, Billy Bob Thornton, Cate Blanch..."
3,Le Seigneur des anneaux : La Communauté de l'a...,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...,https://www.themoviedb.org/movie/120/cast,"[Elijah Wood, Ian McKellen, Viggo Mortensen, S..."
4,Charlotte Gray,https://fr.wikipedia.org/wiki/Charlotte_Gray,https://www.themoviedb.org/movie/12660/cast,"[Cate Blanchett, Billy Crudup, Michael Gambon,..."
5,Terre Neuve (film),https://fr.wikipedia.org/wiki/Terre_Neuve_(film),https://www.themoviedb.org/movie/6440/cast,"[Kevin Spacey, Julianne Moore, Cate Blanchett,..."
6,"Heaven (film, 2002)","https://fr.wikipedia.org/wiki/Heaven_(film,_2002)",https://www.themoviedb.org/movie/10575/cast,"[Cate Blanchett, Giovanni Ribisi, Remo Girone,..."
7,Le Seigneur des anneaux : Les Deux Tours,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...,https://www.themoviedb.org/movie/121/cast,"[Elijah Wood, Ian McKellen, Viggo Mortensen, S..."
8,Veronica Guerin (film),https://fr.wikipedia.org/wiki/Veronica_Guerin_...,https://www.themoviedb.org/movie/10629/cast,"[Cate Blanchett, Gerard McSorley, Ciarán Hinds..."
9,Coffee and Cigarettes,https://fr.wikipedia.org/wiki/Coffee_and_Cigar...,https://www.themoviedb.org/movie/883/cast,"[Roberto Benigni, Steven Wright, Joie Lee, Cin..."


3. Utiliser le résultat de la question précédente pour répondre à la question initiale : avec quels acteurs Cate Blanchett a-t-elle partagé l'affiche le plus souvent au cours des années 2000 ?

In [19]:
# Mise en commun de tous des noms des acteurs
acteurs_list = [
    acteur
    for acteurs in films.acteurs.to_list()
    for acteur in acteurs
]

# La réponse vient en comptant les occurrences de chaque acteur.
# Nous retrouvons bien les acteurs de la trilogie du Seigneur des Anneaux :-)
(
    pd.Series(acteurs_list)
    .value_counts()
    .head(20)
)

Cate Blanchett      23
Peter Jackson        4
Hugo Weaving         4
Billy Boyd           3
Sala Baker           3
Jørn Benzon          3
Craig Parker         3
Billy Jackson        3
Andy Serkis          3
Orlando Bloom        3
Lee Hartley          3
Dominic Monaghan     3
John Rhys-Davies     3
Sam Kelly            3
Liv Tyler            3
Ian Holm             3
Sean Astin           3
Viggo Mortensen      3
Ian McKellen         3
Elijah Wood          3
dtype: int64